In [2]:
import os
import pandas as pd
import janitor

pef = os.path.join('..', 'data', '2022.xlsx')

pef_df=pd.read_excel(pef)
pef_df= pef_df.clean_names()


In [3]:
pd.DataFrame(pef_df.dtypes)

,0
ciclo,int64
id_ramo,int64
desc_ramo,object
id_ur,object
desc_ur,object
gpo_funcional,int64
desc_gpo_funcional,object
id_funcion,int64
desc_funcion,object
id_subfuncion,int64


### Gráfica por institución

In [17]:
import ipywidgets as widgets
import plotly.graph_objects as go

# Lista de desc_ramo
ramos = pef_df['desc_ramo'].unique().tolist()

# Widget Dropdown
ramo_dropdown = widgets.Dropdown(
    options=ramos,
    value=ramos[0],
    description='Ramo:',
)

# Función para generar gráfico
def plot_graph(ramo):
    ramo_df = pef_df[pef_df['desc_ramo'] == ramo]
    ramo_df = ramo_df.groupby('desc_ur').agg(
        monto_aprobado = ('monto_aprobado', 'sum'),
        monto_pagado = ('monto_pagado', 'sum')
    ).sort_values('monto_aprobado', ascending=True)  # Agrega sort_values aquí

    # Creando las barras para el Monto Aprobado
    trace1 = go.Bar(
        y=ramo_df.index,  # eje Y
        x=ramo_df['monto_aprobado'],  # eje X
        name='Monto Aprobado',
        orientation='h',  # Cambia la orientación a horizontal
        marker_color='blue'
    )

    # Creando las barras para el Monto Pagado
    trace2 = go.Bar(
        y=ramo_df.index,  # eje Y
        x=ramo_df['monto_pagado'],  # eje X
        name='Monto Pagado',
        orientation='h',  # Cambia la orientación a horizontal
        marker_color='green'
    )

    # Agregamos las barras en la gráfica
    data = [trace1, trace2]

    # Creamos el layout
    layout = go.Layout(
        title=f'{ramo} - Comparación entre el Monto Aprobado y el Monto Pagado',
        barmode='group',  # permite agrupar las barras en vez de superponerlas
        xaxis=dict(
            title='Monto',
            tickformat=","  # Agrega formato de comas a las marcas del eje X
        ),
        yaxis=dict(title='Concepto', automargin=True),  # automargin ayuda a ajustar el margen automáticamente para el texto de etiqueta
        plot_bgcolor='white',  # Cambia el color de fondo del plot
        paper_bgcolor='white',  # Cambia el color de fondo del paper
        font=dict(
            size=14,  # Cambia el tamaño de la fuente
            color="black"  # Cambia el color de la fuente
        )
    )

    # Creamos el objeto Figure y visualizamos
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(
        separators=".,",  # Ajusta el formato de los números a español
        width=1200, 
        height=1200  # Agrega aquí las dimensiones de la gráfica
    )
    fig.show()

# Función para manejar el evento de cambio en el Dropdown
def on_ramo_change(change):
    plot_graph(change.new)

# Asociamos la función 'on_ramo_change' al evento 'value' del Dropdown
ramo_dropdown.observe(on_ramo_change, names='value')

# Mostramos el Dropdown
display(ramo_dropdown)

# Generamos el primer gráfico
plot_graph(ramos[0])


Dropdown(description='Ramo:', options=('Instituto Nacional Electoral', 'Comisión Nacional de los Derechos Huma…

In [20]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource, CustomJS, SelectMultiple
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.palettes import Category20
output_notebook()

# Lista de desc_ramo
ramos = pef_df['desc_ramo'].unique().tolist()

# Ramo seleccionado
ramo = ramos[0]

# Subconjunto del DataFrame para el ramo seleccionado
df_ramo = pef_df[pef_df['desc_ramo'] == ramo]

# Lista de desc_ur para el ramo seleccionado
desc_ur_list = df_ramo['desc_ur'].unique().tolist()

# DataFrame agrupado por desc_ur
grouped_df = df_ramo.groupby('desc_ur').agg(
    monto_aprobado=('monto_aprobado', 'sum'),
    monto_pagado=('monto_pagado', 'sum')
).sort_values('monto_aprobado', ascending=True)

# Convertimos el DataFrame a un ColumnDataSource de Bokeh
source = ColumnDataSource(data=grouped_df)

# Creamos una figura
p = figure(y_range=grouped_df.index.tolist(), plot_height=800, plot_width=800, 
           title=f"{ramo} - Comparación entre el Monto Aprobado y el Monto Pagado", 
           x_range=(0, grouped_df[['monto_aprobado', 'monto_pagado']].max().max()*1.1))

# Añadimos las barras al gráfico
for i, desc_ur in enumerate(desc_ur_list):
    source_i = ColumnDataSource(data=grouped_df.loc[grouped_df.index == desc_ur])
    p.hbar_stack(['monto_aprobado', 'monto_pagado'], y='desc_ur', height=0.8, color=Category20[20][i*2:i*2+2], 
                 source=source_i, legend_label=desc_ur)

# Creamos el widget SelectMultiple
select = SelectMultiple(options=desc_ur_list, value=desc_ur_list, height=400)

# Código JavaScript que se ejecutará cuando cambie el valor del SelectMultiple
callback = CustomJS(args=dict(p=p, select=select, source=source, df=grouped_df, desc_ur_list=desc_ur_list), code="""
    // Obtenemos las desc_ur seleccionadas
    var selected_ur = select.value;
    
    // Obtenemos los datos originales
    var data = df.data;
    
    // Filtramos los datos
    var new_data = {};
    for (var key in data) {
        new_data[key] = [];
        for (var i = 0; i < data['index'].length; i++) {
            if (selected_ur.includes(data['index'][i])) {
                new_data[key].push(data[key][i]);
            }
        }
    }
    
    // Actualizamos el source
    source.data = new_data;
    
    // Actualizamos el y_range de la figura
    p.y_range.factors = new_data['index'];
    
    // Actualizamos el x_range de la figura
    var max_val = Math.max(...new_data['monto_aprobado'], ...new_data['monto_pagado']);
    p.x_range.start = 0;
    p.x_range.end = max_val * 1.1;
""")

# Asociamos el callback al SelectMultiple
select.js_on_change('value', callback)

# Mostramos el gráfico y el widget
layout = row(p, select)
show(layout, notebook_handle=True)


ImportError: cannot import name 'SelectMultiple' from 'bokeh.models' (/Users/arturoherrera/anaconda3/envs/cuenta_publica/lib/python3.9/site-packages/bokeh/models/__init__.py)

In [19]:
pip install bokeh

     |████████████████████████████████| 7.8 MB 1.6 MB/s eta 0:00:01
  Using cached contourpy-1.1.0-cp39-cp39-macosx_11_0_arm64.whl (229 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl (174 kB)
     |████████████████████████████████| 56 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
